In [ ]:
# 📓 Notebook: Rule-Based Country Assignment (RBCA) for Avian Influenza FASTA files
# --- 1. Load required packages ---
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import re
import os
import time
import json
import datetime
import zipfile
import glob
from collections import Counter, defaultdict
from tqdm import tqdm

# --- 2. User configuration ---
fasta_folder = "."
fasta_files = sorted(glob.glob(os.path.join(fasta_folder, "*.fasta")), key=os.path.getmtime, reverse=True)

if fasta_files:
    fasta_path = fasta_files[0]
    print(f"✅ Auto-selected the latest FASTA file：{fasta_path}")
else:
    raise FileNotFoundError("❗ No .fasta files found. Please check the directory.")
location_to_country_path = "location_to_country_ISO_3166_1.json"

# --- 3. Initialize location-country mapping ---
if os.path.exists(location_to_country_path):
    with open(location_to_country_path, "r", encoding="utf-8") as f:
        location_to_country = json.load(f)
    print(f"✅ Loaded {len(location_to_country)} location-to-country entries.")
else:
    location_to_country = {}
    print("⚡ Not found location_to_country_ISO_3166_1.json. Creating and saving an empty dictionary.")
    try:
        with open(location_to_country_path, "w", encoding="utf-8") as f:
            json.dump(location_to_country, f, ensure_ascii=False, indent=4)
        print(f"✅ Created and saved an empty {location_to_country_path}")
    except Exception as e:
        print(f"❌ Failed to save location_to_country_ISO_3166_1.json error: {e}.")

# --- 4. Parse location from FASTA headers ---
with open(fasta_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Extract only header lines (starting with '>')
headers = [line.strip() for line in lines if line.startswith(">")]
location_list = []
fail_count = 0
fail_headers = []

# Use regex to extract location name（The substring between the second and third slashes）
for header in headers:
    match = re.search(r"A/[^/]+/([^/]+)/", header)
    if match:
        location = match.group(1).strip()
        location_list.append(location)
    else:
        fail_count += 1
        fail_headers.append(header)
        
print(f"📋 Total raw FASTA header sequences: {len(headers)}. Successfully extracted locations: {len(location_list)}. Error: {fail_count}.")

# 📢 Display list of error headers
if fail_headers:
    print("⚠️ The headers of the virus strains that caused errors are listed below:")
    for fh in fail_headers:
        print(f"  - {fh}")

# --- 📂 5. Read location list ---
# Count the occurrence of each location name
location_counter = Counter(location_list)
total_samples = sum(location_counter.values())

# Convert to DataFrame, sort by sample count, and save as CSV
location_df = pd.DataFrame(location_counter.items(), columns=["Location", "Count"]).sort_values(by="Count", ascending=False)
location_df.to_csv("location_list.csv", index=False)
print(f"📄 Saved location list to location_list.csv (Total of {len(location_df)} entries).")

# --- 🌍 6. Sample count by country
country_counter = defaultdict(int)
other_locations = defaultdict(int)
for loc, count in location_counter.items():
    country = location_to_country.get(loc, "Other")
    if country == "Other":
        other_locations[loc] += count
    country_counter[country] += count

country_df = pd.DataFrame(sorted(country_counter.items(), key=lambda x: x[1], reverse=True), columns=["Country", "Sample_Count"])

# Check for consistency in total sample count
assert country_df["Sample_Count"].sum() == total_samples, f"Inconsistent sample total!！Total: {country_df['Sample_Count'].sum()}. It should be: {total_samples}"

# Save country statistics as CSV
country_df.to_csv("country_stat.csv", index=False)

# 📢 Task completion report
print(f"🏁 All country counting completed! Total number of successfully extracted locations: {total_samples}. Sum of country counts: {country_df['Sample_Count'].sum()} — Perfect match ✅")

# 📢 Recognition results
success_samples = total_samples - sum(other_locations.values())
fail_samples = sum(other_locations.values())

print(f"🌐 Successfully identified {success_samples} samples using ISO-3166-1 country codes. {fail_samples} samples could not be matched and were classified as 'Other'. Top 20 unmatched location names:")
if other_locations:
    # Extract the top 20 unrecognized location names by sample count
    for loc, count in sorted(other_locations.items(), key=lambda x: x[1], reverse=True)[:20]:
        print(f"  - {loc}: {count}")

# --- 🖼️ 7. Generate bar chart of top 20 countries by sample count
country_df = pd.read_csv("country_stat.csv")

top20_countries = country_df.head(20)

# Plot the graph
plt.figure(figsize=(14, 8))
bars = plt.barh(top20_countries['Country'], top20_countries['Sample_Count'], color='steelblue')
plt.xlabel('Sample Count', fontsize=13)
plt.title('Top 20 Countries by Sample Count', fontsize=15, pad=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.gca().invert_yaxis()

# Label each bar with the corresponding value
for bar in bars:
    width = bar.get_width()
    plt.text(width + (0.01 * top20_countries['Sample_Count'].max()),
             bar.get_y() + bar.get_height()/2,
             f'{int(width)}', 
             va='center', ha='left', fontsize=11)

plt.tight_layout()

# Save as high-resolution PNG
plt.savefig("top20_countries_sample_count.png", dpi=600)
plt.show()

print("🖼️ Saved bar chart with sample count annotations:top20_countries_sample_count.png")